In [1]:
import pandas as pd
from functools import reduce

In [2]:
up_2005 = pd.read_csv("../data/up_gp_sarpanch_2005_fixed.csv", low_memory = False)
up_2010 = pd.read_csv("../data/up_gp_sarpanch_2010_fixed.csv", low_memory = False)

In [3]:
# Load transliteration
up_2005_trans = pd.read_csv("../data/transliteration/up_gp_sarpanch_2005_transliterate_out.csv", low_memory = False)
up_2010_trans = pd.read_csv("../data/transliteration/up_gp_sarpanch_2010_transliterate_out.csv", low_memory = False)

In [4]:
district_translit = pd.read_csv("../data/transliteration/district_official_hindi_english.csv")
block_translit = pd.read_csv("../data/transliteration/block_name_transliteration.csv")

In [5]:
# Let's convert to dict
mapping_dict_1 = up_2005_trans.set_index('Name')['Transliterated'].str.strip().to_dict()
mapping_dict_2 = up_2010_trans.set_index('Name')['Transliterated'].str.strip().to_dict()
mapping_dict_3 = district_translit.set_index('hindi')['eng'].str.strip().to_dict()
mapping_dict_4 = block_translit.set_index('hindi')['eng'].str.strip().to_dict()

merged_dict = reduce(lambda x, y: x | y, [mapping_dict_1, mapping_dict_2, mapping_dict_3, mapping_dict_4])
len(merged_dict)

79628

In [6]:
up_2005['gp_name_eng'] = up_2005['gp_name_fin'].str.strip().map(merged_dict)
up_2010['gp_name_eng'] = up_2010['gp_name_fin'].str.strip().map(merged_dict)

In [7]:
print(up_2005['gp_name_eng'].notna().sum())

50211


In [8]:
up_2005['district_name_eng'] = up_2005['district_name'].str.strip().map(merged_dict)
up_2010['district_name_eng'] = up_2010['district_name'].str.strip().map(merged_dict)

In [9]:
print(up_2005['district_name_eng'].notna().sum())
print(up_2010['district_name_eng'].notna().sum())

51879
51861


In [10]:
up_2005['block_name_eng'] = up_2005['block_name'].str.strip().map(merged_dict)
up_2010['block_name_eng'] = up_2010['block_name'].str.strip().map(merged_dict)

In [11]:
print(up_2005['block_name_eng'].notna().sum())
print(up_2010['block_name_eng'].notna().sum())

51879
51415


In [12]:
up_2005['elected_sarpanch_name_eng'] = up_2005['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()
up_2005['elected_sarpanch_name_eng'].notna().sum()

42690

In [13]:
up_2010['elected_sarpanch_name_eng'] = up_2010['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()
up_2010['elected_sarpanch_name_eng'].notna().sum()

51688

In [14]:
up_2010['husband_spouse_name_eng'] = up_2010['husband_spouse_name'].str.strip().map(merged_dict).str.strip()
up_2010['husband_spouse_name_eng'].notna().sum()

51787

In [15]:
up_2005.to_csv("../data/up_gp_sarpanch_2005_fixed_with_transliteration.csv", index = False)
up_2010.to_csv("../data/up_gp_sarpanch_2010_fixed_with_transliteration.csv", index = False)